# Phase One: Data Collection & Cleaning
## Portfolio Management PPI Modeling Project

**Objective**: Collect and clean data for forecasting month-on-month percentage changes in PPI for portfolio management services (PPIDF01 Index).

**Key Data Sources**:
- Primary Target: PPIDF01 Index (PPI Portfolio Management, NSA) from FRED
- Market Returns: S&P 500, NASDAQ, Russell 2000, VIX
- Bond Markets: Treasury yields, corporate spreads
- Macro Indicators: Dollar index, commodities, employment data

**Technical Stack**: Polars for data manipulation, FRED API, caching system

## Setup and Configuration

In [1]:
import os
import sys
from pathlib import Path
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import our data collection module
from data_collection import DataCollector

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Environment setup complete!")
print(f"Polars version: {pl.__version__}")
print(f"Current working directory: {os.getcwd()}")

Environment setup complete!
Polars version: 1.31.0
Current working directory: /Users/shashankshankar/Desktop/Business/Capstone Projects/Portfolio Management Modeling


## API Key Configuration

**Important**: You need to get a FRED API key from https://fred.stlouisfed.org/docs/api/api_key.html

1. Create a `.env` file in the project root
2. Add your API key: `FRED_API_KEY=your_key_here`

In [ ]:
# Check if FRED API key is configured
fred_key = os.getenv('FRED_API_KEY')
if fred_key:
    print("FRED API key configured")
    print(f"Key starts with: {fred_key[:8]}...")
else:
    print("WARNING: FRED API key not found!")
    print("Please set FRED_API_KEY in your .env file")
    print("Get your key from: https://fred.stlouisfed.org/docs/api/api_key.html")

## Initialize Data Collector

In [3]:
# Initialize the data collector
collector = DataCollector(cache_dir="data_cache")

print("Data collector initialized with caching enabled")
print(f"Cache directory: {collector.cache_dir}")

Data collector initialized with caching enabled
Cache directory: data_cache


## Step 1: Collect Target Variable
### PPIDF01 Index - PPI Portfolio Management (NSA)

In [4]:
# Fetch the primary target variable
try:
    target_data = collector.get_target_variable()
    print(f"Target data shape: {target_data.shape}")
    print(f"Date range: {target_data['date'].min()} to {target_data['date'].max()}")
    print("\nFirst few rows:")
    print(target_data.head())
    
    # Basic statistics
    print("\nBasic statistics:")
    print(target_data.select(pl.col('value').describe()))
    
except Exception as e:
    print(f"Error fetching target data: {e}")
    target_data = None

Fetching target variable: PPI for Portfolio Management Services
Target data shape: (229, 3)
Date range: 2003-12-01 to 2022-12-01

First few rows:
shape: (5, 3)
┌────────────┬───────┬───────────────┐
│ date       ┆ value ┆ series_id     │
│ ---        ┆ ---   ┆ ---           │
│ date       ┆ f64   ┆ str           │
╞════════════╪═══════╪═══════════════╡
│ 2003-12-01 ┆ 100.0 ┆ PCU5239252392 │
│ 2004-01-01 ┆ 105.1 ┆ PCU5239252392 │
│ 2004-02-01 ┆ 105.2 ┆ PCU5239252392 │
│ 2004-03-01 ┆ 104.9 ┆ PCU5239252392 │
│ 2004-04-01 ┆ 107.4 ┆ PCU5239252392 │
└────────────┴───────┴───────────────┘

Basic statistics:
Error fetching target data: 'Expr' object has no attribute 'describe'


## Step 2: Collect Market Returns Data

In [5]:
# Fetch market returns data
try:
    market_data = collector.get_market_returns_data()
    
    print("Market data collected:")
    for name, df in market_data.items():
        if isinstance(df, pl.DataFrame):
            print(f"  {name}: {df.shape} - Range: {df['date'].min()} to {df['date'].max()}")
    
    # Show sample data
    if 'SP500' in market_data:
        print("\nS&P 500 sample data:")
        print(market_data['SP500'].head())
        
except Exception as e:
    print(f"Error fetching market data: {e}")
    market_data = {}

Fetching SP500 data...
Fetching NASDAQCOM data...
Fetching RUT data...
Fetching VIXCLS data...
Market data collected:
  SP500: (2514, 3) - Range: 2015-07-20 to 2025-07-17
  NASDAQCOM: (6424, 3) - Range: 2000-01-03 to 2025-07-17
  VIXCLS: (6449, 3) - Range: 2000-01-03 to 2025-07-17

S&P 500 sample data:
shape: (5, 3)
┌────────────┬─────────┬───────────┐
│ date       ┆ value   ┆ series_id │
│ ---        ┆ ---     ┆ ---       │
│ date       ┆ f64     ┆ str       │
╞════════════╪═════════╪═══════════╡
│ 2015-07-20 ┆ 2128.28 ┆ SP500     │
│ 2015-07-21 ┆ 2119.21 ┆ SP500     │
│ 2015-07-22 ┆ 2114.15 ┆ SP500     │
│ 2015-07-23 ┆ 2102.15 ┆ SP500     │
│ 2015-07-24 ┆ 2079.65 ┆ SP500     │
└────────────┴─────────┴───────────┘


## Step 3: Collect Bond Market Data

In [6]:
# Fetch bond market data
try:
    bond_data = collector.get_bond_market_data()
    
    print("Bond market data collected:")
    for name, df in bond_data.items():
        if isinstance(df, pl.DataFrame):
            print(f"  {name}: {df.shape} - Range: {df['date'].min()} to {df['date'].max()}")
    
    # Show sample data
    if 'DGS10' in bond_data:
        print("\n10-Year Treasury sample data:")
        print(bond_data['DGS10'].head())
        
except Exception as e:
    print(f"Error fetching bond data: {e}")
    bond_data = {}

Fetching DGS10 data...
Fetching DGS2 data...
Fetching DGS3MO data...
Fetching BAMLC0A0CM data...
Bond market data collected:
  DGS10: (6387, 3) - Range: 2000-01-03 to 2025-07-16
  DGS2: (6387, 3) - Range: 2000-01-03 to 2025-07-16
  DGS3MO: (6387, 3) - Range: 2000-01-03 to 2025-07-16
  BAMLC0A0CM: (6667, 3) - Range: 2000-01-03 to 2025-07-17

10-Year Treasury sample data:
shape: (5, 3)
┌────────────┬───────┬───────────┐
│ date       ┆ value ┆ series_id │
│ ---        ┆ ---   ┆ ---       │
│ date       ┆ f64   ┆ str       │
╞════════════╪═══════╪═══════════╡
│ 2000-01-03 ┆ 6.58  ┆ DGS10     │
│ 2000-01-04 ┆ 6.49  ┆ DGS10     │
│ 2000-01-05 ┆ 6.62  ┆ DGS10     │
│ 2000-01-06 ┆ 6.57  ┆ DGS10     │
│ 2000-01-07 ┆ 6.52  ┆ DGS10     │
└────────────┴───────┴───────────┘


## Step 4: Collect Macro Indicators

In [7]:
# Fetch macro indicators
try:
    macro_data = collector.get_macro_indicators()
    
    print("Macro indicators collected:")
    for name, df in macro_data.items():
        if isinstance(df, pl.DataFrame):
            print(f"  {name}: {df.shape} - Range: {df['date'].min()} to {df['date'].max()}")
    
    # Show sample data
    if 'DTWEXBGS' in macro_data:
        print("\nDollar Index sample data:")
        print(macro_data['DTWEXBGS'].head())
        
except Exception as e:
    print(f"Error fetching macro data: {e}")
    macro_data = {}

Fetching DTWEXBGS data...
Fetching DCOILWTICO data...
Fetching GOLDAMGBD228NLBM data...
Fetching UNRATE data...
Fetching CPIAUCSL data...
Macro indicators collected:
  DTWEXBGS: (4895, 3) - Range: 2006-01-02 to 2025-07-11
  DCOILWTICO: (6402, 3) - Range: 2000-01-04 to 2025-07-14
  UNRATE: (306, 3) - Range: 2000-01-01 to 2025-06-01
  CPIAUCSL: (306, 3) - Range: 2000-01-01 to 2025-06-01

Dollar Index sample data:
shape: (5, 3)
┌────────────┬──────────┬───────────┐
│ date       ┆ value    ┆ series_id │
│ ---        ┆ ---      ┆ ---       │
│ date       ┆ f64      ┆ str       │
╞════════════╪══════════╪═══════════╡
│ 2006-01-02 ┆ 101.4155 ┆ DTWEXBGS  │
│ 2006-01-03 ┆ 100.7558 ┆ DTWEXBGS  │
│ 2006-01-04 ┆ 100.2288 ┆ DTWEXBGS  │
│ 2006-01-05 ┆ 100.2992 ┆ DTWEXBGS  │
│ 2006-01-06 ┆ 100.0241 ┆ DTWEXBGS  │
└────────────┴──────────┴───────────┘


## Step 5: Data Quality Assessment

In [8]:
def assess_data_quality(df: pl.DataFrame, name: str) -> dict:
    """Assess data quality for a given DataFrame."""
    if df is None or df.height == 0:
        return {"name": name, "status": "empty", "issues": ["No data available"]}
    
    issues = []
    
    # Check for missing values
    null_count = df.null_count().sum_horizontal()[0]
    if null_count > 0:
        issues.append(f"Missing values: {null_count}")
    
    # Check date range
    date_range = df['date'].max() - df['date'].min()
    if date_range.days < 365:
        issues.append(f"Short time series: {date_range.days} days")
    
    # Check for duplicates
    duplicate_count = df.height - df.unique().height
    if duplicate_count > 0:
        issues.append(f"Duplicate rows: {duplicate_count}")
    
    return {
        "name": name,
        "rows": df.height,
        "date_range": f"{df['date'].min()} to {df['date'].max()}",
        "issues": issues if issues else ["No issues detected"]
    }

# Assess all collected data
quality_report = []

if target_data is not None:
    quality_report.append(assess_data_quality(target_data, "PPIDF01 (Target)"))

for category, data_dict in [("Market", market_data), ("Bond", bond_data), ("Macro", macro_data)]:
    for name, df in data_dict.items():
        if isinstance(df, pl.DataFrame):
            quality_report.append(assess_data_quality(df, f"{category}: {name}"))

# Display quality report
print("DATA QUALITY ASSESSMENT")
print("=" * 50)
for report in quality_report:
    print(f"\n{report['name']}:")
    print(f"  Rows: {report.get('rows', 'N/A')}")
    print(f"  Range: {report.get('date_range', 'N/A')}")
    print(f"  Issues: {', '.join(report['issues'])}")

DATA QUALITY ASSESSMENT

Market: SP500:
  Rows: 2514
  Range: 2015-07-20 to 2025-07-17
  Issues: No issues detected

Market: NASDAQCOM:
  Rows: 6424
  Range: 2000-01-03 to 2025-07-17
  Issues: No issues detected

Market: VIXCLS:
  Rows: 6449
  Range: 2000-01-03 to 2025-07-17
  Issues: No issues detected

Bond: DGS10:
  Rows: 6387
  Range: 2000-01-03 to 2025-07-16
  Issues: No issues detected

Bond: DGS2:
  Rows: 6387
  Range: 2000-01-03 to 2025-07-16
  Issues: No issues detected

Bond: DGS3MO:
  Rows: 6387
  Range: 2000-01-03 to 2025-07-16
  Issues: No issues detected

Bond: BAMLC0A0CM:
  Rows: 6667
  Range: 2000-01-03 to 2025-07-17
  Issues: No issues detected

Macro: DTWEXBGS:
  Rows: 4895
  Range: 2006-01-02 to 2025-07-11
  Issues: No issues detected

Macro: DCOILWTICO:
  Rows: 6402
  Range: 2000-01-04 to 2025-07-14
  Issues: No issues detected

Macro: UNRATE:
  Rows: 306
  Range: 2000-01-01 to 2025-06-01
  Issues: No issues detected

Macro: CPIAUCSL:
  Rows: 306
  Range: 2000-01-01

## Step 6: Initial Data Visualization

In [9]:
# Plot target variable if available
if target_data is not None and target_data.height > 0:
    # Convert to pandas for plotting
    target_pd = target_data.to_pandas()
    target_pd['date'] = pd.to_datetime(target_pd['date'])
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Time series plot
    ax1.plot(target_pd['date'], target_pd['value'], linewidth=2, color='navy')
    ax1.set_title('PPI Portfolio Management (PPIDF01) - NSA', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Index Value')
    ax1.grid(True, alpha=0.3)
    
    # Calculate month-over-month percentage change
    target_pd['mom_pct'] = target_pd['value'].pct_change() * 100
    
    # MoM percentage change plot
    ax2.plot(target_pd['date'], target_pd['mom_pct'], linewidth=1.5, color='darkred', alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax2.set_title('Month-over-Month Percentage Change (Target Variable)', fontsize=14, fontweight='bold')
    ax2.set_ylabel('MoM % Change')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics for MoM changes
    mom_stats = target_pd['mom_pct'].describe()
    print("\nMonth-over-Month % Change Statistics:")
    print(mom_stats)
    
else:
    print("Target data not available for visualization")

Target data not available for visualization


## Step 7: Save Collected Data

In [ ]:
# Compile all data into a single structure
all_collected_data = {
    'target': target_data,
    'market_returns': market_data,
    'bond_market': bond_data,
    'macro_indicators': macro_data
}

# Save to file
try:
    collector.save_collected_data(all_collected_data, "phase_one_collected_data.json")
    print("All data saved successfully")
except Exception as e:
    print(f"Error saving data: {e}")

# Create summary report
summary = {
    'collection_date': datetime.now().isoformat(),
    'target_variable': 'PPIDF01 - PPI Portfolio Management (NSA)',
    'data_sources': {
        'target': 1 if target_data is not None else 0,
        'market_returns': len(market_data),
        'bond_market': len(bond_data),
        'macro_indicators': len(macro_data)
    },
    'total_series': 1 + len(market_data) + len(bond_data) + len(macro_data)
}

print("\nphase one completion summary")
print("=" * 40)
print(f"collection date: {summary['collection_date']}")
print(f"target variable: {summary['target_variable']}")
print(f"total series collected: {summary['total_series']}")
print("\ndata sources:")
for source, count in summary['data_sources'].items():
    print(f"  {source.replace('_', ' ').title()}: {count} series")

print("\nphase one: data collection strategy - complete")
print("next: phase two - exploratory data analysis")